读取数据，切分x，y  
jieba分词  
词频处理  
算法处理

In [1]:
import os
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib # 存储模型
from sklearn.metrics import classification_report, confusion_matrix # 分类报告、混淆矩阵
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 传入文件路径，获取特征和标签
def read_data(file_dir):
    x = []
    y = []
    with open(file_dir, encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        # 找到每行数据切分特征、标签
#         y_, x_ = line.split('---')
        # unicode 唯一编码
        # 将数据进行编解码
        line = line.encode('unicode-escape').decode('unicode-escape')
        l = line.split('---')
        if len(l) != 2:
            continue
        y_, x_ = l
        x_ = x_[:-2] # 切掉\n
        # 获取特征后，进行jieba分词处理
        x_ = jieba.lcut(x_)
        x.append(' '.join(x_))
        y.append(y_)
    return x, y

In [3]:
# 获取训练集的特征和标签
# 查看文件夹中的所有文件
dir_list = os.listdir('train_data')
# 存储获取的特征和标签
x_train, y_train = [], []
for file_dir in dir_list:
    x, y = read_data('train_data/' + file_dir)
    x_train += x
    y_train += y
# 验证长度
print(len(x_train), len(y_train))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.037 seconds.
Prefix dict has been built successfully.


10399 10399


In [4]:
x_train[:5]

['1 - 10 月 全国 房地产 开发 投资 同比 增长 6.3% ， 增速 提高 0.7 个 百分',
 '1 - 10 月 河北省 外贸 进出口 保持 增长   同比 增长 6',
 '1.1 万亿 美债 被 减后 ， 美媒 ： 中国 或 清零 美债 ， 美 或 发生 债务 危',
 '10 - 11 月 中国 巧克力 品牌 线上 发展 排行榜 单 TOP1',
 '10000 美金 的 天价 海运费 来 了 ！ 订舱 之前 先摇号 ， 一舱 难求']

In [5]:
# 获取停用词的信息  
# 默认gbk
with open('stop_word/stopword.txt', encoding='utf-8') as f:
    lines = f.readlines()
# 存储列表
stop_words = []
for line in lines:
    line = line.encode('unicode-escape').decode('unicode-escape')
    stop_words.append(line.rstrip('\n'))
stop_words[:5]

['--', '?', '“', '”', '》']

In [6]:
# 词频统计
cv = CountVectorizer(stop_words=stop_words) # 将停用词带入
x_train = cv.fit_transform(x_train).toarray()

In [7]:
print('存储的单词量', len(cv.get_feature_names()))

存储的单词量 23533


In [8]:
# 模型处理
model = MultinomialNB().fit(x_train, y_train)
print('模型的准确率为（训练集）', model.score(x_train, y_train))

模型的准确率为（训练集） 0.9920184633137802


测试集得分

In [9]:
dir_list = os.listdir('test_data')
# 存储获取的特征和标签
x_test, y_test = [], []
for file_dir in dir_list:
    x, y = read_data('test_data/' + file_dir)
    x_test += x
    y_test += y

In [10]:
# 词频统计
x_test = cv.transform(x_test).toarray()

In [11]:
model.score(x_test, y_test)

0.961082910321489

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
# 朴素贝叶斯处理文本数据效果好、运行速度快，准确率高，不需要调参
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf.score(x_test, y_test)

0.8578680203045685

In [14]:
# 保存模型
joblib.dump(model, 'nb.pkl') # 朴素贝叶斯
joblib.dump(cv, 'cv.pkl') # 词频统计

['cv.pkl']

In [15]:
model.predict(x_test[:1])

array(['财经'], dtype='<U2')

In [16]:
y_test[:1]

['财经']

In [17]:
x_test.shape

(591, 23533)